# LoRA Serving

SGLang enables the use of [LoRA adapters](https://arxiv.org/abs/2106.09685) with a base model. By incorporating techniques from [S-LoRA](https://arxiv.org/pdf/2311.03285) and [Punica](https://arxiv.org/pdf/2310.18547), SGLang can efficiently support multiple LoRA adapters for different sequences within a single batch of inputs.

## Arguments for LoRA Serving

The following server arguments are relevant for multi-LoRA serving:

* `lora_paths`: A mapping from each adaptor's name to its path, in the form of `{name}={path} {name}={path}`.

* `max_loras_per_batch`: Maximum number of adaptors used by each batch. This argument can affect the amount of GPU memory reserved for multi-LoRA serving, so it should be set to a smaller value when memory is scarce. Defaults to be 8.

* `lora_backend`: The backend of running GEMM kernels for Lora modules. It can be one of `triton` or `flashinfer`, and set to `triton` by default. For better performance and stability, we recommend using the Triton LoRA backend. In the future, faster backend built upon Cutlass or Cuda kernels will be added.

* `max_lora_rank`: The maximum LoRA rank that should be supported. If not specified, it will be automatically inferred from the adapters provided in `--lora-paths`. This argument is needed when you expect to dynamically load adapters of larger LoRA rank after server startup.

* `lora_target_modules`: The union set of all target modules where LoRA should be applied (e.g., `q_proj`, `k_proj`, `gate_proj`). If not specified, it will be automatically inferred from the adapters provided in `--lora-paths`. This argument is needed when you expect to dynamically load adapters of different target modules after server startup.

* `tp_size`: LoRA serving along with Tensor Parallelism is supported by SGLang. `tp_size` controls the number of GPUs for tensor parallelism. More details on the tensor sharding strategy can be found in [S-Lora](https://arxiv.org/pdf/2311.03285) paper.

From client side, the user needs to provide a list of strings as input batch, and a list of adaptor names that each input sequence corresponds to.

## Usage

### Serving Single Adaptor

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, terminate_process

import json
import requests

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    --max-loras-per-batch 1 --lora-backend triton \
    --disable-radix-cache
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-15 08:01:38] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=37933, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_outp

[2025-07-15 08:01:50] Attention backend not set. Use fa3 backend by default.
[2025-07-15 08:01:50] Init torch distributed begin.


[2025-07-15 08:01:51] Init torch distributed ends. mem usage=0.00 GB


[2025-07-15 08:01:53] Load weight begin. avail mem=78.46 GB


[2025-07-15 08:01:53] Using model weights format ['*.safetensors']


...

[2025-07-15 08:01:57] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=43.32 GB


[2025-07-15 08:01:58] INFO:     Started server process [3590398]
[2025-07-15 08:01:58] INFO:     Waiting for application startup.
[2025-07-15 08:01:58] INFO:     Application startup complete.
[2025-07-15 08:01:58] INFO:     Uvicorn running on http://127.0.0.1:37933 (Press CTRL+C to quit)


[2025-07-15 08:01:59] INFO:     127.0.0.1:40506 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-15 08:01:59] INFO:     127.0.0.1:40508 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-15 08:01:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:01:59.510161


[2025-07-15 08:02:00] INFO:     127.0.0.1:40516 - "POST /generate HTTP/1.1" 200 OK
[2025-07-15 08:02:00] The server is fired up and ready to roll!


In [3]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "AI is a field of computer science focused on",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses the base model
    "lora_path": ["lora0", None],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-07-15 08:02:04] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:02:04.304815


[2025-07-15 08:02:05] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.44, #queue-req: 1, timestamp: 2025-07-15T08:02:05.300523
[2025-07-15 08:02:05] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:02:05.300746


[2025-07-15 08:02:06] INFO:     127.0.0.1:40526 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  creating intelligent machines that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI has many applications in various


In [4]:
terminate_process(server_process)

[2025-07-15 08:02:06] Child process unexpectedly failed with exitcode=9. pid=3591252


### Serving Multiple Adaptors

In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    lora1=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16 \
    --max-loras-per-batch 2 --lora-backend triton \
    --disable-radix-cache
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-15 08:02:17] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=32306, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_outp

[2025-07-15 08:02:28] Attention backend not set. Use fa3 backend by default.
[2025-07-15 08:02:28] Init torch distributed begin.


[2025-07-15 08:02:29] Init torch distributed ends. mem usage=0.00 GB


[2025-07-15 08:02:31] Load weight begin. avail mem=57.96 GB


[2025-07-15 08:02:32] Using model weights format ['*.safetensors']


...

[2025-07-15 08:02:37] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=23.46 GB


[2025-07-15 08:02:38] INFO:     Started server process [3593520]
[2025-07-15 08:02:38] INFO:     Waiting for application startup.
[2025-07-15 08:02:38] INFO:     Application startup complete.
[2025-07-15 08:02:38] INFO:     Uvicorn running on http://127.0.0.1:32306 (Press CTRL+C to quit)


[2025-07-15 08:02:38] INFO:     127.0.0.1:52932 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-15 08:02:39] INFO:     127.0.0.1:52934 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-15 08:02:39] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:02:39.013635


[2025-07-15 08:02:41] INFO:     127.0.0.1:52940 - "POST /generate HTTP/1.1" 200 OK
[2025-07-15 08:02:41] The server is fired up and ready to roll!


In [6]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "AI is a field of computer science focused on",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-07-15 08:02:43] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:02:43.286798
[2025-07-15 08:02:43] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-15T08:02:43.288271


[2025-07-15 08:02:44] Decode batch. #running-req: 2, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 9.89, #queue-req: 0, timestamp: 2025-07-15T08:02:44.732468
[2025-07-15 08:02:44] INFO:     127.0.0.1:52956 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  creating intelligent machines capable of performing tasks that typically require human intelligence. AI research has led to the development of various techniques, including machine learning, natural language processing,


In [7]:
terminate_process(server_process)

[2025-07-15 08:02:44] Child process unexpectedly failed with exitcode=9. pid=3594373
[2025-07-15 08:02:44] Child process unexpectedly failed with exitcode=9. pid=3593887


### Dynamic LoRA loading

### Basic Usage

Instead of specifying all adapters during server startup via `--lora-paths`. You can also load & unload LoRA adapters dynamically via the `/load_lora_adapter` and `/unload_lora_adapter` API.

(Please note that, currently we still require you to specify at least one adapter in `--lora-paths` to enable the LoRA feature, this limitation will be lifted soon.)

In [8]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --lora-paths lora0=philschmid/code-llama-3-1-8b-text-to-sql-lora \
    --cuda-graph-max-bs 2 \
    --max-loras-per-batch 2 --lora-backend triton \
    --disable-radix-cache
    """
)

url = f"http://127.0.0.1:{port}"
wait_for_server(url)

[2025-07-15 08:02:56] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=37053, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_outp

[2025-07-15 08:03:07] Attention backend not set. Use fa3 backend by default.
[2025-07-15 08:03:07] Init torch distributed begin.


[2025-07-15 08:03:07] Init torch distributed ends. mem usage=1.07 GB


[2025-07-15 08:03:08] Load weight begin. avail mem=42.71 GB


[2025-07-15 08:03:09] Using model weights format ['*.safetensors']


...

[2025-07-15 08:03:13] LoRA manager ready. Loaded LoRA adapters: lora0
[2025-07-15 08:03:13] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-07-15 08:03:13] Memory pool end. avail mem=20.89 GB


[2025-07-15 08:03:14] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=20.79 GB


[2025-07-15 08:03:14] INFO:     Started server process [3596650]
[2025-07-15 08:03:14] INFO:     Waiting for application startup.
[2025-07-15 08:03:14] INFO:     Application startup complete.
[2025-07-15 08:03:14] INFO:     Uvicorn running on http://127.0.0.1:37053 (Press CTRL+C to quit)
[2025-07-15 08:03:14] INFO:     127.0.0.1:40728 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-15 08:03:15] INFO:     127.0.0.1:40730 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-15 08:03:15] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:03:15.990808


[2025-07-15 08:03:17] INFO:     127.0.0.1:40738 - "POST /generate HTTP/1.1" 200 OK
[2025-07-15 08:03:17] The server is fired up and ready to roll!


In [9]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora1",
        "lora_path": "Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16",
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-07-15 08:03:19] Start load Lora adapter. Lora name=lora1, path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16
[2025-07-15 08:03:19] LoRA adapter loading starts: name=lora1, path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16. avail mem=56.89 GB
[2025-07-15 08:03:19] Loading weight of LoRA adapter lora1 from Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16
[2025-07-15 08:03:20] Using model weights format ['*.safetensors']


[2025-07-15 08:03:20] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 91.69it/s]

[2025-07-15 08:03:20] LoRA adapter loading completes: name=lora1, path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16. avail mem=56.89 GB
[2025-07-15 08:03:20] Cache flushed successfully!
[2025-07-15 08:03:20] INFO:     127.0.0.1:45756 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora0': 'philschmid/code-llama-3-1-8b-text-to-sql-lora', 'lora1': 'Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16'}}


In [10]:
response = requests.post(
    url + "/generate",
    json={
        "text": [
            "List 3 countries and their capitals.",
            "List 3 countries and their capitals.",
        ],
        "sampling_params": {"max_new_tokens": 32, "temperature": 0},
        "lora_path": ["lora0", "lora1"],
    },
)
print(f"Output from lora0: {response.json()[0]['text']}")
print(f"Output from lora1: {response.json()[1]['text']}")

[2025-07-15 08:03:20] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:03:20.184697
[2025-07-15 08:03:20] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-15T08:03:20.185778


[2025-07-15 08:03:22] INFO:     127.0.0.1:45764 - "POST /generate HTTP/1.1" 200 OK
Output from lora0:  Country 1 has a capital of Bogor? No, that's not correct. The capital of Country 1 is actually Bogor is not the capital,
Output from lora1:  Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -


In [11]:
response = requests.post(
    url + "/unload_lora_adapter",
    json={
        "lora_name": "lora0",
    },
)

[2025-07-15 08:03:22] Start unload Lora adapter. Lora name=lora0
[2025-07-15 08:03:22] LoRA adapter unloading starts: name=lora0. avail mem=56.88 GB
[2025-07-15 08:03:22] Unloading LoRA adapter lora0
[2025-07-15 08:03:22] LoRA adapter unloading completes: name=lora0. avail mem=56.88 GB
[2025-07-15 08:03:22] Cache flushed successfully!
[2025-07-15 08:03:22] INFO:     127.0.0.1:45776 - "POST /unload_lora_adapter HTTP/1.1" 200 OK


In [12]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora2",
        "lora_path": "pbevan11/llama-3.1-8b-ocr-correction",
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-07-15 08:03:22] Start load Lora adapter. Lora name=lora2, path=pbevan11/llama-3.1-8b-ocr-correction
[2025-07-15 08:03:22] LoRA adapter loading starts: name=lora2, path=pbevan11/llama-3.1-8b-ocr-correction. avail mem=56.88 GB
[2025-07-15 08:03:22] Loading weight of LoRA adapter lora2 from pbevan11/llama-3.1-8b-ocr-correction


[2025-07-15 08:03:22] Using model weights format ['*.safetensors']


[2025-07-15 08:03:22] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 90.03it/s]

[2025-07-15 08:03:22] LoRA adapter loading completes: name=lora2, path=pbevan11/llama-3.1-8b-ocr-correction. avail mem=56.88 GB
[2025-07-15 08:03:22] Cache flushed successfully!
[2025-07-15 08:03:22] INFO:     127.0.0.1:45786 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora1': 'Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16', 'lora2': 'pbevan11/llama-3.1-8b-ocr-correction'}}


In [13]:
response = requests.post(
    url + "/generate",
    json={
        "text": [
            "List 3 countries and their capitals.",
            "List 3 countries and their capitals.",
        ],
        "sampling_params": {"max_new_tokens": 32, "temperature": 0},
        "lora_path": ["lora1", "lora2"],
    },
)
print(f"Output from lora1: {response.json()[0]['text']}")
print(f"Output from lora2: {response.json()[1]['text']}")

[2025-07-15 08:03:22] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:03:22.484546
[2025-07-15 08:03:22] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-15T08:03:22.485558


[2025-07-15 08:03:23] INFO:     127.0.0.1:45796 - "POST /generate HTTP/1.1" 200 OK
Output from lora1:  Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -
Output from lora2:  1.  Japan 2.  China 3.  India  Capital of Japan:  Tokyo Capital of China:  Beijing Capital of India:


In [14]:
terminate_process(server_process)

[2025-07-15 08:03:23] Child process unexpectedly failed with exitcode=9. pid=3597425


### Advanced: hosting adapters of different shapes

In some cases, you may want to load LoRA adapters with different ranks or target modules (e.g., `q_proj`, `k_proj`) simultaneously. To ensure the server can accommodate all expected LoRA shapes, it's recommended to explicitly specify `--max-lora-rank` and/or `--lora-target-modules` at startup.

For backward compatibility, SGLang will infer these values from `--lora-paths` if they are not explicitly provided. This means it's safe to omit them **only if** all dynamically loaded adapters share the same shape (rank and target modules) as those in the initial `--lora-paths` or are strictly "smaller".

In [15]:
lora0 = "Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16"  # rank - 4, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj
lora1 = "algoprog/fact-generation-llama-3.1-8b-instruct-lora"  # rank - 64, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj


# The `--target-lora-modules` param below is technically not needed, as the server will infer it from lora0 which already has all the target modules specified.
# We are adding it here just to demonstrate usage.
server_process, port = launch_server_cmd(
    f"""
    python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --lora-paths lora0={lora0} \
    --cuda-graph-max-bs 2 \
    --max-loras-per-batch 2 --lora-backend triton \
    --disable-radix-cache
    --max-lora-rank 64
    --lora-target-modules q_proj k_proj v_proj o_proj down_proj up_proj gate_proj
    """
)

url = f"http://127.0.0.1:{port}"
wait_for_server(url)

[2025-07-15 08:03:34] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=32267, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_outp

[2025-07-15 08:03:47] Attention backend not set. Use fa3 backend by default.
[2025-07-15 08:03:47] Init torch distributed begin.


[2025-07-15 08:03:47] Init torch distributed ends. mem usage=0.00 GB


[2025-07-15 08:03:48] Load weight begin. avail mem=60.49 GB


[2025-07-15 08:03:49] Using model weights format ['*.safetensors']


...

[2025-07-15 08:03:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=41.86 GB


[2025-07-15 08:03:54] INFO:     Started server process [3599921]
[2025-07-15 08:03:54] INFO:     Waiting for application startup.
[2025-07-15 08:03:54] INFO:     Application startup complete.
[2025-07-15 08:03:54] INFO:     Uvicorn running on http://127.0.0.1:32267 (Press CTRL+C to quit)


[2025-07-15 08:03:54] INFO:     127.0.0.1:53578 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-15 08:03:55] INFO:     127.0.0.1:53594 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-15 08:03:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:03:55.247251


[2025-07-15 08:03:59] INFO:     127.0.0.1:53604 - "POST /generate HTTP/1.1" 200 OK
[2025-07-15 08:03:59] The server is fired up and ready to roll!


In [16]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora1",
        "lora_path": lora1,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-07-15 08:03:59] Start load Lora adapter. Lora name=lora1, path=algoprog/fact-generation-llama-3.1-8b-instruct-lora
[2025-07-15 08:03:59] LoRA adapter loading starts: name=lora1, path=algoprog/fact-generation-llama-3.1-8b-instruct-lora. avail mem=41.45 GB
[2025-07-15 08:03:59] Loading weight of LoRA adapter lora1 from algoprog/fact-generation-llama-3.1-8b-instruct-lora


[2025-07-15 08:03:59] Using model weights format ['*.safetensors']
[2025-07-15 08:03:59] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 119.63it/s]

[2025-07-15 08:03:59] Gate projection base_model.model.model.layers.0.mlp.gate_proj.lora_A.weight does not have a corresponding up projection base_model.model.model.layers.0.mlp.up_proj.lora_A.weight. Initializing up projection to zero.
[2025-07-15 08:03:59] Gate projection base_model.model.model.layers.0.mlp.gate_proj.lora_B.weight does not have a corresponding up projection base_model.model.model.layers.0.mlp.up_proj.lora_B.weight. Initializing up projection to zero.
[2025-07-15 08:04:00] Gate projection base_model.model.model.layers.1.mlp.gate_proj.lora_A.weight does not have a corresponding up projection base_model.model.model.layers.1.mlp.up_proj.lora_A.weight. Initializing up project

[2025-07-15 08:04:00] Gate projection base_model.model.model.layers.13.mlp.gate_proj.lora_A.weight does not have a corresponding up projection base_model.model.model.layers.13.mlp.up_proj.lora_A.weight. Initializing up projection to zero.
[2025-07-15 08:04:00] Gate projection base_model.model.model.layers.13.mlp.gate_proj.lora_B.weight does not have a corresponding up projection base_model.model.model.layers.13.mlp.up_proj.lora_B.weight. Initializing up projection to zero.
[2025-07-15 08:04:00] Gate projection base_model.model.model.layers.14.mlp.gate_proj.lora_A.weight does not have a corresponding up projection base_model.model.model.layers.14.mlp.up_proj.lora_A.weight. Initializing up projection to zero.
[2025-07-15 08:04:00] Gate projection base_model.model.model.layers.14.mlp.gate_proj.lora_B.weight does not have a corresponding up projection base_model.model.model.layers.14.mlp.up_proj.lora_B.weight. Initializing up projection to zero.
[2025-07-15 08:04:00] Gate projection base_m

In [17]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "AI is a field of computer science focused on",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0: {response.json()[0]['text']}")
print(f"Output from lora1: {response.json()[1]['text']}")

[2025-07-15 08:04:00] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-15T08:04:00.241067
[2025-07-15 08:04:00] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-15T08:04:00.242088


[2025-07-15 08:04:01] INFO:     127.0.0.1:44708 - "POST /generate HTTP/1.1" 200 OK
Output from lora0:  Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -
Output from lora1:  creating intelligent machines that can perform tasks that typically require human intelligence. AI is a broad field that encompasses many subfields, including machine learning, natural language processing,


In [18]:
terminate_process(server_process)

[2025-07-15 08:04:01] Child process unexpectedly failed with exitcode=9. pid=3601152


## Future Works

The development roadmap for LoRA-related features can be found in this [issue](https://github.com/sgl-project/sglang/issues/2929). Currently radix attention is incompatible with LoRA and must be manually disabled. Other features, including Unified Paging, Cutlass backend, and dynamic loading/unloadingm, are still under development.